In [1]:
# A couple of ways to import and slice-dice the fixture list using the premier league API

#imports
import numpy as np
import pandas as pd
import requests
import datetime as dt
import xlwings as xw

In [2]:
#pick up the fixture list and the static data (fixtures uses team ids) from the api. Basically lots of JSON files 
url = 'https://fantasy.premierleague.com/api/fixtures/'
r = requests.get(url)
fixtures_json = r.json()

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
static_json = r.json()


In [3]:
#from the team static set up a dictionary of team id to team name
teams = static_json["teams"]
team_codes ={team["id"]:team["name"] for team in teams}


In [4]:
#Loop through the fixtures and put in a dataframe. Can't parse the date as it has nulls which causes problems here
fixture_list = [[team_codes[fixture['team_h']],team_codes[fixture['team_a']],fixture['kickoff_time']] for fixture in fixtures_json]


In [5]:
# Attempt 1 - filter the list above by team

#define the team you want
team = "Liverpool"

#tidy up the dataframe and parse the dates (works beter outside the loop)    
dfFixtures = pd.DataFrame(fixture_list,columns=["Home","Away","Date/Time"])
dfFixtures["Date/Time"] = pd.to_datetime(dfFixtures["Date/Time"],format="%Y-%m-%dT%H:%M:%SZ")

teamFixtures = dfFixtures[(dfFixtures["Home"]==team) | (dfFixtures["Away"]==team)]

display(teamFixtures.style.hide_index())

### output to excel. commented out.
#sht = xw.Book().sheets[0]
#sht.range('A1').options(index=False).value = teamFixtures


Home,Away,Date/Time
Liverpool,Leeds,2020-09-12 16:30:00
Chelsea,Liverpool,2020-09-20 15:30:00
Liverpool,Arsenal,2020-09-28 19:00:00
Aston Villa,Liverpool,2020-10-03 14:00:00
Everton,Liverpool,2020-10-17 14:00:00
Liverpool,Sheffield Utd,2020-10-24 14:00:00
Liverpool,West Ham,2020-10-31 15:00:00
Man City,Liverpool,2020-11-07 15:00:00
Liverpool,Leicester,2020-11-21 15:00:00
Brighton,Liverpool,2020-11-28 15:00:00


In [6]:
# Attempt 2 - create a custom output format and iterate over the fixture list 
#define the team you want
team = "Spurs"

#first define the function for the custom output text
def format_fixture(x,team):
    home=x[0]
    away=x[1]
    date = dt.datetime.strptime(x[2],"%Y-%m-%dT%H:%M:%SZ")
    if home ==team:
        return ["{}".format(dt.datetime.strftime(date,"%a %d-%b"))," {}(H) {}".format(away,dt.datetime.strftime(date,"%H:%M"))]
    elif away ==team:
        return ["{}".format(dt.datetime.strftime(date,"%a %d-%b"))," {}(A) {}".format(home,dt.datetime.strftime(date,"%H:%M"))]

#conditonal formatting function    
def home_bold(val):
    bold = 'bold' if "(H)" in val  else ''
    return 'font-weight:{}'.format(bold)
    
#then put through some pythonic loops to get a dataframe of date and fixture 

output = [format_fixture(x,team) for x in fixture_list[2:] if format_fixture(x,team) != None]

#print ditching the index and using contitional format
pd.DataFrame(output,columns=["Date","Fixture"]).style.hide_index().applymap(home_bold)

Date,Fixture
Sun 13-Sep,Everton(H) 15:30
Sun 20-Sep,Southampton(A) 11:00
Sat 26-Sep,Newcastle(H) 14:00
Sat 03-Oct,Man Utd(A) 14:00
Sat 17-Oct,West Ham(H) 14:00
Sat 24-Oct,Burnley(A) 14:00
Sat 31-Oct,Brighton(H) 15:00
Sat 07-Nov,West Brom(A) 15:00
Sat 21-Nov,Man City(H) 15:00
Sat 28-Nov,Chelsea(A) 15:00
